In [92]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.figure_factory as ff
import json
import geojson
import requests
import plotly.graph_objs as go
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#%matplotlib notebook
pd.options.display.max_columns = 150
pd.options.display.max_rows = 2000



# Insertion du dataset Pricing

In [93]:
price = pd.read_csv("prices.csv", sep=',', encoding='utf-8')
price['jour'] = pd.to_datetime(price['price creation date'], errors='coerce').dt.strftime('%Y-%m-%d')
price['month'] = pd.to_datetime(price['price creation date'], errors='coerce').dt.strftime('%Y-%m')
price['year'] = pd.to_datetime(price['price creation date'], errors='coerce').dt.strftime('%Y')
price = price.rename(columns={'price value':'price_value'})
price.sample(2)

,Unnamed: 0,id,uid,boost_id,name,lat,lng,address,postal_code,city_name,phone,email,since,price_name,price category,price_label,price_value,price_name.1,price_year,price creation date,opening_hours_concat,activities,status_1,status_2,jour,month,year
11126,11126,7615,5572147d743142db,NaN,Cimetière Communal de Pouilly-sur-Serre,49.674975,3.586784,D635,2270,Pouilly-sur-Serre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60987,60987,42496,7ba399236a880f87,NaN,Pompes Funèbres Marbrerie Monuroc,49.314586,6.125917,Rue des Artisans,57190,Florange,382587770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-08:30-18:00|2-08:30-18:00|3-08:30-18:00|4-08...,undertaker,0.0,0.0,NaN,NaN,NaN


# ajout des départements avec les noms dans le dataset

In [94]:
# je récupère le numéro du département à 2 chiffres (attention aux 10 premiers départements)
def get_department(postal_code):
    postal_code = str(postal_code)
    return postal_code[:2]

# Traitement des CP incomplets
price['postal_code'] = price['postal_code'].fillna('98').astype(str)
price['postal_code'] = price['postal_code'].str.zfill(5)
price['dpt_pf'] = price['postal_code'].apply(lambda x : get_department(x))

# Je rajoute la BDD des départements pour insérer les noms des départements
departements_info = pd.read_excel('departements-francais.xls')
number_to_departement = departements_info.set_index('NUMÉRO')['NOM'].to_dict()

#Ajout du nom du département dans le dataset
def convert_dpt(price):
    
    price['dpt_pf'] = price['dpt_pf'].apply(lambda x : x.lstrip('0'))
    missing_dpt_number = price[price['dpt_pf'] == ''].shape[0]
    print(f'Dropping {missing_dpt_number} out of {price.shape[0]} leads because of missing departement number')
    price = price[price['dpt_pf'] != '']
    price['dpt_pf'] = price['dpt_pf'].astype(int)
    price['departement'] = price['dpt_pf'].apply(lambda x : number_to_departement.get(x, ""))
    
    return price

price = convert_dpt(price)

# je modifie les noms des départements qui ne sont pas correctes
price['departement'] = price['departement'].str.replace("Val-D'Oise", "Val-d'Oise")
price['departement'] = price['departement'].str.replace( 'Seine-St-Denis', "Seine-Saint-Denis")
price['departement'] = price['departement'].str.replace( "Côtes d'Armor", "Côtes-d'Armor")
price['departement'] = price['departement'].str.replace( "Vandée", "Vendée")
price.sample(1)

Dropping 0 out of 48820 leads because of missing departement number


,Unnamed: 0,id,uid,boost_id,name,lat,lng,address,postal_code,city_name,phone,email,since,price_name,price category,price_label,price_value,price_name.1,price_year,price creation date,opening_hours_concat,activities,status_1,status_2,jour,month,year,dpt_pf,departement
52471,52471,40676,10938143b3926ac5,NaN,Pompes Funèbres Garcia J.P.,43.456961,1.323827,"66, Avenue des Pyrénées",31600,Muret,561873584.0,jean-pierre.garcia201@orange.fr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,undertaker,0.0,0.0,NaN,NaN,NaN,31,Haute-Garonne


# DATA Cimetière nationnal

In [95]:
# focus que sur les cimetières
price=price.fillna({'activities':'cimetiere'})
price_cim=price[price['activities']=='cimetiere']
print('nombre de tarifs de cimetières',price_cim.shape)
price_cim.sample(10)

nombre de tarifs de cimetières (48820, 29)


,Unnamed: 0,id,uid,boost_id,name,lat,lng,address,postal_code,city_name,phone,email,since,price_name,price category,price_label,price_value,price_name.1,price_year,price creation date,opening_hours_concat,activities,status_1,status_2,jour,month,year,dpt_pf,departement
18677,18677,14346,8740914f8f9c0cf4,NaN,Cimetière Ancien D'aulnay-sous-Bois,48.942008,2.504700,109-111 Route de Mitry,93600,Aulnay-sous-Bois,NaN,sg@aulnaysousbois.com,NaN,Case columbarium 30 ans,Colombarium,columbarium-case-30years,513.0,Cimetière Ancien D'aulnay-sous-Bois,2021.0,2020-11-18 07:35:26.619664,NaN,cimetiere,NaN,NaN,2020-11-18,2020-11,2020,93,Seine-Saint-Denis
2271,2271,932,10b66f9838e172fb,NaN,Cimetière Communal,48.049920,-1.868546,15 Rue de Montfort,35310,Bréal-sous-Montfort,NaN,mairie@brealsousmontfort.fr,NaN,Concession cinéraire 30 ans,Concession cinéraire,cinerary-concession-30years,1240.0,Cimetière Communal,2021.0,2021-11-17 08:09:15.546065,NaN,cimetiere,NaN,NaN,2021-11-17,2021-11,2021,35,Ille-et-Vilaine
20288,20288,15304,3d061d285069b067,NaN,Cimetière Communal de Saint-Martin-de-Crau,43.636974,4.799137,Avenue Marcel Pagnol,13310,Saint-Martin-de-Crau,490471729.0,mairie@stmartindecrau.fr,NaN,Concession 30 ans 2 places,Concession Cimetières,concession-30years-2places,960.0,Cimetière Communal de Saint-Martin-de-Crau,2021.0,2020-09-30 06:22:52.556841,NaN,cimetiere,NaN,NaN,2020-09-30,2020-09,2020,13,Bouches-du-Rhône
24562,24562,18385,a4a86bdcfa3b0570,NaN,Cimetière Communal de le Gouray,48.328442,-2.491377,6542 Rue de Croquelien,22330,Le Gouray,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,22,Côtes-d'Armor
29651,29651,22100,4f5dcc4e04d43a01,NaN,Cimetière Communal de la Guérinière,46.966487,-2.233201,1 Rue de la Rampe,85680,La Guérinière,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,85,Vendée
25388,25388,19090,5ff083d90036187c,NaN,Cimetière Communal de le Thuit,49.257897,1.359028,4 Rue du Chancelier Maupéou,27700,Le Thuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,27,Eure
28702,28702,21489,7adf167e3ac08d9c,NaN,Cimetière Communal de Niederbronn-les-Bains,48.947277,7.651235,47-49 Route de Reichshoffen,67110,Niederbronn-les-Bains,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,67,Bas-Rhin
34795,34795,25517,3e1494de1588dd46,NaN,Cimetière Communal de Mazières-en-Mauges,47.048994,-0.813779,D200,49280,Mazières-en-Mauges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,49,Maine-et-Loire
23350,23350,17314,a2792cb6c3601994,NaN,Cimetière Communal D'orval,46.724724,2.481840,1 Route de Lignières,18200,Orval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,18,Cher
37550,37550,28066,09a6b7bc2fcf601f,NaN,Cimetière Communal de Corbehem,50.338725,3.045589,13 Rue de la Gare,62112,Corbehem,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cimetiere,NaN,NaN,NaN,NaN,NaN,62,Pas-de-Calais


# choix de l'agrégat= 30 ans
il faudrait pouvoir rajouter d'autres agrégats comme 15 ans

In [96]:
# selection des champs 15 ans
price_cim_15 = price_cim[price_cim.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]

# selection des champs 30 ans
price_cim_30 = price_cim[price_cim.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]

# selection des champs 50 ans
price_cim_50 = price_cim[price_cim.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]

# data carte de france avec départements

In [97]:
# 15 ans
price_cim_15_dpt=price_cim_15[['departement','name','price_label','postal_code','price_value']]
price_cim_15_dpt_mean=price_cim_15_dpt.groupby(['departement']).mean()[['price_value']]
price_cim_15_dpt_mean=round(price_cim_15_dpt_mean,1).reset_index()
price_cim_15_dpt_mean.sample(2)

# 30 ans
price_cim_30_dpt=price_cim_30[['departement','name','price_label','postal_code','price_value']]
price_cim_30_dpt_mean=price_cim_30_dpt.groupby(['departement']).mean()[['price_value']]
price_cim_30_dpt_mean=round(price_cim_30_dpt_mean,1).reset_index()
price_cim_30_dpt_mean.sample(2)

# 50 ans
price_cim_50_dpt=price_cim_50[['departement','name','price_label','postal_code','price_value']]
price_cim_50_dpt_mean=price_cim_50_dpt.groupby(['departement']).mean()[['price_value']]
price_cim_50_dpt_mean=round(price_cim_50_dpt_mean,1).reset_index()
price_cim_50_dpt_mean.sample(2)

,departement,price_value
56,Meuse,586.0
62,Paris,3343.5


In [64]:
# Fichier à télécharger ici : https://github.com/gregoiredavid/france-geojson/blob/master/departements.geojson

with open ("geojson/departements.geojson", "r") as f:
    departements = geojson.load(f)

with open ("geojson/villes_principales_france.geojson", "r") as f2:
    villes_principales = geojson.load(f2)
villes_principales = villes_principales.features

# on définit la fonction qui génère la carte selon le dataset donné
def generateDepartementsMap(dataSet, fileName, years):
    attribute = 'price_value'
    
    # on récupère les départements non présents
    deps=[]
    for i, d in dataSet.iterrows():
        deps.append(d['departement'])
    for dpt in departements.features:
        if dpt["properties"]["nom"] not in deps:
            dataSet = dataSet.append({"departement": dpt["properties"]["nom"], "price_value": 0}, ignore_index = True)
            print('missing departement:', dpt["properties"]["nom"])
            
    # on définit la fig de la carte à générer
    fig = px.choropleth(
        dataSet,
        geojson=departements,
        locations='departement',
        scope='europe',
        color=dataSet[attribute],
        range_color=(dataSet[attribute].min(), dataSet[attribute].max()),
        featureidkey="properties.nom",
        hover_data=[attribute],
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'departement': 'Département'}
    )
    
    # on affiche les villes principales sur la fig avec le geojson chargé précédemment
    for ville in villes_principales:
        fig.add_trace(go.Scattergeo(
            lon = [ville.geometry.coordinates[0]],
            lat = [ville.geometry.coordinates[1]],
            text = ville.properties['nom'],
            textfont=dict(
                size=12,
                color="white"
            ),
            mode="text",
            showlegend = False,
            hoverinfo="skip"
        ))
        
    fig.update_layout(title_text = "Prix moyen des concessions dans les départements de France sur " + years + " ans")
    fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))       
    fig.update_geos(fitbounds="locations", visible=False)
    
    # génération du fichier de la carte donnée
    fig.write_html(fileName)
    fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateDepartementsMap(price_cim_15_dpt_mean, "cartes-inter/15y-concessions-departements.html", "15")
#generateDepartementsMap(price_cim_30_dpt_mean, "cartes-inter/30y-concessions-departements.html", "30")
#generateDepartementsMap(price_cim_50_dpt_mean, "cartes-inter/50y-concessions-departements.html", "50")

# le fichier cartes-inter/concessions-departements.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

missing departement: Corse-du-Sud
missing departement: Haute-Corse
missing departement: Haute-Loire
missing departement: Lot
missing departement: Lozère
missing departement: Paris
missing departement: Somme
missing departement: Territoire de Belfort
missing departement: Corse-du-Sud
missing departement: Haute-Corse
missing departement: Indre
missing departement: Haute-Loire
missing departement: Lozère
missing departement: Somme
missing departement: Corse-du-Sud
missing departement: Haute-Corse
missing departement: Ariège
missing departement: Dordogne
missing departement: Indre
missing departement: Haute-Loire
missing departement: Lozère
missing departement: Orne
missing departement: Haut-Rhin
missing departement: Somme


# data carte de france avec communes

In [98]:
# selection des datasets pertinents
price_cim_15=price_cim_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]
price_cim_15.sample(1)

price_cim_30=price_cim_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]
price_cim_30.sample(1)

price_cim_50=price_cim_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]
price_cim_50.sample(1)

,departement,city_name,name,price_label,lat,lng,postal_code,price_value
90158,Deux-Sèvres,Bressuire,Cimetière rue St-Simon,concession-50years-1m2,46.844052,-0.497548,79300,176.0


In [ ]:
'''# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('geojson/communes-France.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateCommunesMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    #df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)

    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )

    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )

    fig.add_trace(go.Scattergeo(
            lon = df[df['lng'] != 0.0]['lng'],
            lat = df[df['lng'] != 0.0]['lat'],
            text = df[df['lng'] != 0.0]['full_text'],
            #textfont=dict(size=12,
            #    color="red"
            #),
            mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes de France sur ' + years + ' ans')

    fig.update_geos(fitbounds="locations", visible=False)

    # génération du fichier de la carte donnée
    fig.write_html(fileName)

    fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateCommunesMap(price_cim_15, "cartes_inter/15y-concessions-communes.html", "15")
#generateCommunesMap(price_cim_30, "cartes_inter/30y-concessions-communes.html", "30")
#generateCommunesMap(price_cim_50, "cartes_inter/50y-concessions-communes.html", "50")

# le fichier cartes_inter/concessions-communes.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus'''

# DATA Cimetière département

In [99]:
# select the right department
price_cim_var=price_cim[price_cim['departement']=='Var']
print(price_cim_var.price_label.count())

# selection des champs 15 ans
price_cim_var_15 = price_cim_var[price_cim_var.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_var_15 = price_cim_var_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_var_30 = price_cim_var[price_cim_var.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_var_30 = price_cim_var_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_var_50 = price_cim_var[price_cim_var.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_var_50 = price_cim_var_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_var_15_mean=price_cim_var_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_15_mean.sample(2)

price_cim_var_30_mean=price_cim_var_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_30_mean.sample(2)

price_cim_var_50_mean=price_cim_var_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_50_mean.sample(2)

507


price_label,city_name,lat,lng,concession-50years-1m2,concession-50years-1place,concession-50years-2m2,concession-50years-2places,concession-50years-3m2,concession-50years-3places,concession-50years-4m2,concession-50years-4places,concession-50years-5m2,concession-50years-5places,concession-50years-6m2,concession-50years-6places
23,Six-Fours-les-Plages,43.089654,5.838408,NaN,NaN,NaN,2375.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,La Môle,43.210573,6.467971,NaN,NaN,NaN,1160.00,NaN,1600.0,NaN,2400.0,NaN,NaN,NaN,3300.0


In [100]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('geojson/communes-83-var.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes du Var')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_var_15, "cartes_inter/15y-concessions-var.html", "15")
generateVarMap(price_cim_var_30, "cartes_inter/30y-concessions-var.html", "30")
generateVarMap(price_cim_var_50, "cartes_inter/50y-concessions-var.html", "50")

# j'appelle la fonction de génération de carte à nouveau pour ajouter l'agrégat que je viens de créer

# le fichier cartes_inter/concessions-departements.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus

# Alpes Maritime

In [ ]:
price_cim.departement.unique()

In [ ]:
# select the right department
price_cim_var=price_cim[price_cim['departement']=='Ille-et-Vilaine']
print(price_cim_var.price_label.count())

# selection des champs 15 ans
price_cim_var_15 = price_cim_var[price_cim_var.price_label.isin(["concession-15years-1m2","concession-15years-2m2","concession-15years-3m2","concession-15years-4m2","concession-15years-5m2","concession-15years-6m2","concession-15years-7m2","concession-15years-8m2","concession-15years-9m2","concession-15years-10m2","concession-15years-1place","concession-15years-2places","concession-15years-3places","concession-15years-4places","concession-15years-5places","concession-15years-6places"])]
price_cim_var_15 = price_cim_var_15[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 30 ans
price_cim_var_30 = price_cim_var[price_cim_var.price_label.isin(["concession-30years-1m2","concession-30years-2m2","concession-30years-3m2","concession-30years-4m2","concession-30years-5m2","concession-30years-6m2","concession-30years-7m2","concession-30years-8m2","concession-30years-9m2","concession-30years-10m2","concession-30years-1place","concession-30years-2places","concession-30years-3places","concession-30years-4places","concession-30years-5places","concession-30years-6places"])]
price_cim_var_30 = price_cim_var_30[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# selection des champs 50 ans
price_cim_var_50 = price_cim_var[price_cim_var.price_label.isin(["concession-50years-1m2","concession-50years-2m2","concession-50years-3m2","concession-50years-4m2","concession-50years-5m2","concession-50years-6m2","concession-50years-7m2","concession-50years-8m2","concession-50years-9m2","concession-50years-10m2","concession-50years-1place","concession-50years-2places","concession-50years-3places","concession-50years-4places","concession-50years-5places","concession-50years-6places"])]
price_cim_var_50 = price_cim_var_50[['departement','city_name','name','price_label','lat','lng','postal_code','price_value']]

# table pour la carte
price_cim_var_15_mean=price_cim_var_15.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_15_mean.sample(2)

price_cim_var_30_mean=price_cim_var_30.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_30_mean.sample(2)

price_cim_var_50_mean=price_cim_var_50.pivot_table(index=['city_name','lat','lng'], columns='price_label', values='price_value', aggfunc='mean').reset_index()
price_cim_var_50_mean.sample(2)

In [ ]:
# https://github.com/gregoiredavid/france-geojson/blob/master/departements/83-var/communes-83-var.geojson

with open('geojson/communes-35-ille-et-vilaine.geojson') as f:
    var_geo = geojson.load(f)

# on définit la fonction qui génère la carte selon le dataset donné
def generateVarMap(dataSet, fileName, years):
    # Read dataset and preprocess
    df = dataSet.groupby(['city_name']).mean()
    city_to_coords = df.to_dict()
    df = df.reset_index()

    city_names = [x['properties']['nom'] for x in var_geo['features']]
    for city in set(city_names):
        if city not in df['city_name']:
            df.loc[-1] = [city, city_to_coords['lat'].get(city, 0), city_to_coords['lng'].get(city, 0), 0]
            df.index = df.index + 1
            df = df.sort_index()

    # code qui permet d'afficher le nom de la ville ou le prix de la concession
    df['full_text'] = df.apply(lambda x : x["city_name"] + '<br>' + str(int(x["price_value"])) + '€', axis=1)
    #df['full_text'] = df.apply(lambda x : str(int(x["price_value"])), axis=1)
    
    # on définit la fig de la carte à générer
    fig = px.choropleth(df, geojson=var_geo, locations='city_name', color='price_value',
        color_continuous_scale="Blues",
        featureidkey='properties.nom',
        labels={'price_value':'Prix moyen sur ' + years + ' ans', 'city_name':'Ville'},
        fitbounds='geojson',
        projection="mercator"
    )
    
    # on garde les 5 communes les moins chères et les 5 communes les plus chères
    for i, d in df.iterrows():
        # on retire les valeurs nulles pour trier les communes dont on connait le prix
        if d.price_value == 0:
            df = df[df.index != i]
    df.sort_values(['price_value'], inplace=True)
    df2 = df.head(5)
    df3 = df.tail(5)
    df = pd.concat([df2, df3])

    # enlever une ville arbitrairement
    # for i, d in df.iterrows():
        # if d.city_name == 'Montferrat':
            # df = df[df.index != i]
    
    fig.update_layout(
        font=dict(
            size=12,
            color="Black"
        )
    )

    # Add images
    '''fig.add_layout_image(
            dict(
                source="stele.jpeg",
                xref="x",
                yref="y",
                x=-0.5,
                y=0.5,
                sizex=1,
                sizey=1,
                opacity=0.5,
                layer="above"
            )
    )'''

    fig.add_trace(go.Scattergeo(
        lon = df[df['lng'] != 0.0]['lng'],
        lat = df[df['lng'] != 0.0]['lat'],
        text = df[df['lng'] != 0.0]['full_text'],
        textfont=dict(
            size=12,
            color="red"
        ),
        mode="text",
    ))

    '''fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
        yaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'
        },
        xaxis = {
            'title':'',
            'showgrid':False,
            'zeroline':False,
            'autorange':'reversed'},
    )'''

    fig.update_layout(title_text = 'Prix moyen des concessions sur les communes du Var')

    fig.update_geos(fitbounds="locations", visible=False)

    # create the html file 
    fig.write_html(fileName)

    # fig.show()
    
# on appelle la fonction qui génère une carte pour le dataset donné en paramètre
generateVarMap(price_cim_var_15, "cartes_inter/15y-concessions-ile-et-vilaine.html", "15")
generateVarMap(price_cim_var_30, "cartes_inter/30y-concessions-ile-et-vilaine.html", "30")
generateVarMap(price_cim_var_50, "cartes_inter/50y-concessions-ile-et-vilaine.html", "50")

# le fichier cartes_inter/concessions-var.html peut désormais être rafraîchi puisqu'il dépend des cartes qu'on génère ci-dessus